In [1]:
import pandas as pd
csv_path = "/kaggle/input/siim-isic-melanoma-classification/train.csv"
df = pd.read_csv(csv_path)
# Shape (rows, columns)
print("Shape of dataset:", df.shape)
print("\nFeatures (columns):")
print(df.columns.tolist())
# Show first 5 rows
print("\nSample rows:")
print(df.head())


Shape of dataset: (33126, 8)

Features (columns):
['image_name', 'patient_id', 'sex', 'age_approx', 'anatom_site_general_challenge', 'diagnosis', 'benign_malignant', 'target']

Sample rows:
     image_name  patient_id     sex  age_approx anatom_site_general_challenge  \
0  ISIC_2637011  IP_7279968    male        45.0                     head/neck   
1  ISIC_0015719  IP_3075186  female        45.0               upper extremity   
2  ISIC_0052212  IP_2842074  female        50.0               lower extremity   
3  ISIC_0068279  IP_6890425  female        45.0                     head/neck   
4  ISIC_0074268  IP_8723313  female        55.0               upper extremity   

  diagnosis benign_malignant  target  
0   unknown           benign       0  
1   unknown           benign       0  
2     nevus           benign       0  
3   unknown           benign       0  
4   unknown           benign       0  


In [2]:
import os
from pathlib import Path

DATASET_DIR = "/kaggle/input/siim-isic-melanoma-classification/jpeg/train"

files = list(Path(DATASET_DIR).glob("*.jpg"))
print("Total image files found:", len(files))
print("First 5 files:", files[:5])


Total image files found: 33126
First 5 files: [PosixPath('/kaggle/input/siim-isic-melanoma-classification/jpeg/train/ISIC_2801761.jpg'), PosixPath('/kaggle/input/siim-isic-melanoma-classification/jpeg/train/ISIC_4442506.jpg'), PosixPath('/kaggle/input/siim-isic-melanoma-classification/jpeg/train/ISIC_0375775.jpg'), PosixPath('/kaggle/input/siim-isic-melanoma-classification/jpeg/train/ISIC_9054603.jpg'), PosixPath('/kaggle/input/siim-isic-melanoma-classification/jpeg/train/ISIC_1691175.jpg')]


In [3]:
import pandas as pd
from pathlib import Path

csv_path = "/kaggle/input/siim-isic-melanoma-classification/train.csv"
df = pd.read_csv(csv_path)
# Add full image path column
DATASET_DIR = "/kaggle/input/siim-isic-melanoma-classification/jpeg/train"
df["image_path"] = df["image_name"].apply(lambda x: str(Path(DATASET_DIR) / f"{x}.jpg"))

print(df.head())


     image_name  patient_id     sex  age_approx anatom_site_general_challenge  \
0  ISIC_2637011  IP_7279968    male        45.0                     head/neck   
1  ISIC_0015719  IP_3075186  female        45.0               upper extremity   
2  ISIC_0052212  IP_2842074  female        50.0               lower extremity   
3  ISIC_0068279  IP_6890425  female        45.0                     head/neck   
4  ISIC_0074268  IP_8723313  female        55.0               upper extremity   

  diagnosis benign_malignant  target  \
0   unknown           benign       0   
1   unknown           benign       0   
2     nevus           benign       0   
3   unknown           benign       0   
4   unknown           benign       0   

                                          image_path  
0  /kaggle/input/siim-isic-melanoma-classificatio...  
1  /kaggle/input/siim-isic-melanoma-classificatio...  
2  /kaggle/input/siim-isic-melanoma-classificatio...  
3  /kaggle/input/siim-isic-melanoma-classificatio...

In [4]:
#1.Imports + Config
import os, shutil, hashlib
from pathlib import Path
from PIL import Image
import numpy as np
import cv2
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, array_to_img, load_img, save_img

# Configuration
CSV_PATH = "/kaggle/input/siim-isic-melanoma-classification/train.csv"
DATASET_DIR = "/kaggle/input/siim-isic-melanoma-classification/jpeg/train"
OUTPUT_DIR = Path("/kaggle/working/output")

IMAGE_SIZE = (224, 224)   

AUGMENT_TO_BALANCE = True
MAX_AUG_PER_IMAGE = 3
TARGET_PER_CLASS = 2000

os.makedirs(OUTPUT_DIR, exist_ok=True)
quarantine_dir = Path(OUTPUT_DIR) / "quarantine"
quarantine_dir.mkdir(parents=True, exist_ok=True)


2025-09-24 08:11:25.790295: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1758701486.040110      13 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1758701486.114418      13 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [5]:
#Link CSV with Images
df = pd.read_csv(CSV_PATH)
df["image_path"] = df["image_name"].apply(lambda x: str(Path(DATASET_DIR) / f"{x}.jpg"))
df["class"] = df["target"].apply(lambda x: "melanoma" if x==1 else "benign")

print("Dataset shape:", df.shape)
print("Classes:", df["class"].value_counts())
df.head()


Dataset shape: (33126, 10)
Classes: class
benign      32542
melanoma      584
Name: count, dtype: int64


,image_name,patient_id,sex,age_approx,anatom_site_general_challenge,diagnosis,benign_malignant,target,image_path,class
0,ISIC_2637011,IP_7279968,male,45.0,head/neck,unknown,benign,0,/kaggle/input/siim-isic-melanoma-classificatio...,benign
1,ISIC_0015719,IP_3075186,female,45.0,upper extremity,unknown,benign,0,/kaggle/input/siim-isic-melanoma-classificatio...,benign
2,ISIC_0052212,IP_2842074,female,50.0,lower extremity,nevus,benign,0,/kaggle/input/siim-isic-melanoma-classificatio...,benign
3,ISIC_0068279,IP_6890425,female,45.0,head/neck,unknown,benign,0,/kaggle/input/siim-isic-melanoma-classificatio...,benign
4,ISIC_0074268,IP_8723313,female,55.0,upper extremity,unknown,benign,0,/kaggle/input/siim-isic-melanoma-classificatio...,benign


In [6]:
#Preprocessing function
def dull_razor_remove_hair(cv_img):
    gray = cv2.cvtColor(cv_img, cv2.COLOR_BGR2GRAY)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (9,9))
    blackhat = cv2.morphologyEx(gray, cv2.MORPH_BLACKHAT, kernel)
    _, thresh = cv2.threshold(blackhat, 10, 255, cv2.THRESH_BINARY)
    dilated = cv2.dilate(thresh, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(3,3)), iterations=1)
    return cv2.inpaint(cv_img, dilated, 1, cv2.INPAINT_TELEA)

def correct_contrast_clahe(cv_img):
    lab = cv2.cvtColor(cv_img, cv2.COLOR_BGR2LAB)
    l,a,b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    l_clahe = clahe.apply(l)
    lab_clahe = cv2.merge((l_clahe,a,b))
    return cv2.cvtColor(lab_clahe, cv2.COLOR_LAB2BGR)

def center_crop(img_pil, scale=1.0):
    w,h = img_pil.size
    new_w, new_h = int(w*scale), int(h*scale)
    left, top = (w-new_w)//2, (h-new_h)//2
    return img_pil.crop((left, top, left+new_w, top+new_h))

def process_and_save(inp_path, out_path, do_hair=True, do_clahe=True, crop_scale=1.0, size=IMAGE_SIZE):
    try:
        img_cv = cv2.imread(str(inp_path))
        if img_cv is None: return False
        if do_hair: img_cv = dull_razor_remove_hair(img_cv)
        if do_clahe: img_cv = correct_contrast_clahe(img_cv)
        img_rgb = cv2.cvtColor(img_cv, cv2.COLOR_BGR2RGB)
        img_pil = Image.fromarray(img_rgb)
        if crop_scale<1.0: img_pil = center_crop(img_pil, crop_scale)
        img_pil = img_pil.resize(size, Image.BILINEAR)
        out_path.parent.mkdir(parents=True, exist_ok=True)
        img_pil.save(out_path, format="JPEG", quality=95)
        return True
    except Exception as e:
        print("Error:", e)
        return False


In [7]:
import cv2
import pandas as pd
from pathlib import Path
from tqdm import tqdm

# Paths
CSV_PATH = "/kaggle/input/siim-isic-melanoma-classification/train.csv"
BASE_IMAGE_DIR = Path("/kaggle/input/siim-isic-melanoma-classification/jpeg/train/")
OUTPUT_DIR = Path("/kaggle/working/output")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# Output dirs
processed_dirs = {
    "original": OUTPUT_DIR / "original",
    "crop75": OUTPUT_DIR / "crop75",
    "crop50": OUTPUT_DIR / "crop50"
}
for p in processed_dirs.values():
    p.mkdir(parents=True, exist_ok=True)

# Checkpoint CSV
CHECKPOINT_FILE = OUTPUT_DIR / "preprocess_progress.csv"
if CHECKPOINT_FILE.exists():
    checkpoint_df = pd.read_csv(CHECKPOINT_FILE)
    processed_images = set(checkpoint_df['image'].tolist())
    print(f"Resuming from checkpoint: {len(processed_images)} already processed")
else:
    processed_images = set()
    checkpoint_df = pd.DataFrame(columns=['image'])
    print("No checkpoint found. Starting fresh.")

# Load image names
df = pd.read_csv(CSV_PATH)
image_names = [f"{x}.jpg" for x in df['image_name']]

def dull_razor_fast(img):
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (17,17))
    blackhat = cv2.morphologyEx(gray, cv2.MORPH_BLACKHAT, kernel)
    _, mask = cv2.threshold(blackhat, 10, 255, cv2.THRESH_BINARY)
    return cv2.inpaint(img, mask, 1, cv2.INPAINT_TELEA)

def process_and_save(img, fname):
    img = dull_razor_fast(img)
    H, W, _ = img.shape
    for scale, dirname in zip([1.0, 0.75, 0.5], ["original", "crop75", "crop50"]):
        new_h, new_w = int(H*scale), int(W*scale)
        start_h, start_w = (H-new_h)//2, (W-new_w)//2
        cropped = img[start_h:start_h+new_h, start_w:start_w+new_w]
        resized = cv2.resize(cropped, (224,224))
        cv2.imwrite(str(processed_dirs[dirname]/fname), cv2.cvtColor(resized, cv2.COLOR_RGB2BGR))

# Process images
for fname in tqdm(image_names, desc="Processing images"):
    if fname in processed_images:  # skip if already done
        continue

    path = BASE_IMAGE_DIR / fname
    img = cv2.imread(str(path))
    if img is None:
        continue

    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    process_and_save(img, fname)

    # Update checkpoint
    checkpoint_df = pd.concat([checkpoint_df, pd.DataFrame({'image': [fname]})], ignore_index=True)
    checkpoint_df.to_csv(CHECKPOINT_FILE, index=False)

print("Preprocessing complete!")


No checkpoint found. Starting fresh.


Processing images: 100%|██████████| 33126/33126 [9:10:53<00:00,  1.00it/s]

Preprocessing complete!


In [8]:
import cv2
import pandas as pd
from pathlib import Path
from tqdm import tqdm

# Paths
CSV_PATH = "/kaggle/input/siim-isic-melanoma-classification/train.csv"
BASE_IMAGE_DIR = Path("/kaggle/input/siim-isic-melanoma-classification/jpeg/train/")
OUTPUT_DIR = Path("/kaggle/working/output")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# Output dirs
processed_dirs = {
    "original": OUTPUT_DIR / "original",
    "crop75": OUTPUT_DIR / "crop75",
    "crop50": OUTPUT_DIR / "crop50"
}
for p in processed_dirs.values():
    p.mkdir(parents=True, exist_ok=True)

# Checkpoint CSV
CHECKPOINT_FILE = OUTPUT_DIR / "preprocess_progress.csv"
if CHECKPOINT_FILE.exists():
    checkpoint_df = pd.read_csv(CHECKPOINT_FILE)
    processed_images = set(checkpoint_df['image'].tolist())
    print(f"Resuming from checkpoint: {len(processed_images)} already processed")
else:
    processed_images = set()
    checkpoint_df = pd.DataFrame(columns=['image'])
    print("No checkpoint found. Starting fresh.")

# Load image names
df = pd.read_csv(CSV_PATH)
image_names = [f"{x}.jpg" for x in df['image_name']]

def dull_razor_fast(img):
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (17,17))
    blackhat = cv2.morphologyEx(gray, cv2.MORPH_BLACKHAT, kernel)
    _, mask = cv2.threshold(blackhat, 10, 255, cv2.THRESH_BINARY)
    return cv2.inpaint(img, mask, 1, cv2.INPAINT_TELEA)

def process_and_save(img, fname):
    img = dull_razor_fast(img)
    H, W, _ = img.shape
    for scale, dirname in zip([1.0, 0.75, 0.5], ["original", "crop75", "crop50"]):
        new_h, new_w = int(H*scale), int(W*scale)
        start_h, start_w = (H-new_h)//2, (W-new_w)//2
        cropped = img[start_h:start_h+new_h, start_w:start_w+new_w]
        resized = cv2.resize(cropped, (224,224))
        cv2.imwrite(str(processed_dirs[dirname]/fname), cv2.cvtColor(resized, cv2.COLOR_RGB2BGR))

# Process images
for fname in tqdm(image_names, desc="Processing images"):
    if fname in processed_images:  # skip if already done
        continue

    path = BASE_IMAGE_DIR / fname
    img = cv2.imread(str(path))
    if img is None:
        continue

    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    process_and_save(img, fname)

    # Update checkpoint
    checkpoint_df = pd.concat([checkpoint_df, pd.DataFrame({'image': [fname]})], ignore_index=True)
    checkpoint_df.to_csv(CHECKPOINT_FILE, index=False)

print("Preprocessing complete!")


Resuming from checkpoint: 33126 already processed


Processing images: 100%|██████████| 33126/33126 [00:00<00:00, 1979406.98it/s]

Preprocessing complete!


In [9]:
from pathlib import Path

# Expected total
TOTAL_IMAGES = 33126  

# Output directory
OUTPUT_DIR = Path("/kaggle/working/output")
processed_dirs = {
    "original": OUTPUT_DIR / "original",
    "crop75": OUTPUT_DIR / "crop75",
    "crop50": OUTPUT_DIR / "crop50"
}

print("🔍 Checking preprocessed images...\n")
all_ok = True

for name, folder in processed_dirs.items():
    count = sum(1 for f in folder.rglob("*.jpg"))
    print(f"{name}: {count}/{TOTAL_IMAGES}")
    if count != TOTAL_IMAGES:
        all_ok = False

if all_ok:
    print("\n✅ Preprocessing completed successfully! All 33126 images exist in every folder.")
else:
    print("\n⚠️ Some images are missing. Preprocessing not fully complete.")


🔍 Checking preprocessed images...

original: 33126/33126
crop75: 33126/33126
crop50: 33126/33126

✅ Preprocessing completed successfully! All 33126 images exist in every folder.


In [10]:
!pip install kaggle
